# HashiCorp Vault Demo for Key Management Secrets Engine 

The Key Management secrets engine provides a consistent workflow for distribution and lifecycle management of cryptographic keys in various key management service (KMS) providers. It allows organizations to maintain centralized control of their keys in Vault while still taking advantage of cryptographic capabilities native to the KMS providers.

The secrets engine generates and owns original copies of key material. When an operator decides to distribute and manage the lifecycle of a key in one of the supported KMS providers, a copy of the key material is distributed. This provides additional durability and disaster recovery means for the complete lifecycle of the key in the KMS provider.

Supported KMS Providers include:
- Azure Key Vault
- AWS KMS
- GCP Cloud KMS

Ref: https://developer.hashicorp.com/vault/docs/secrets/key-management#kms-providers 

Supported Key Types include:
- aes256-gcm96 - AES-GCM with a 256-bit AES key and a 96-bit nonce (symmetric)
- rsa-2048 - RSA with bit size of 2048 (asymmetric)
- rsa-3072 - RSA with bit size of 3072 (asymmetric)
- rsa-4096 - RSA with bit size of 4096 (asymmetric)
- ecdsa-p256 - ECDSA using the P-256 elliptic curve (asymmetric)
- ecdsa-p384 - ECDSA using the P-384 elliptic curve (asymmetric)
- ecdsa-p521 - ECDSA using the P-521 elliptic curve (asymmetric)

Ref: https://developer.hashicorp.com/vault/docs/secrets/key-management#key-types

The following table defines which key types are compatible with each KMS provider.

<img src="images/vault-demo-key-management-kms-keytypes.png">

Ref: https://developer.hashicorp.com/vault/docs/secrets/key-management#compatibility

<img src="images/vault-demo-key-management-kms.png">

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your `VAULT_ADDR` and `VAULT_TOKEN` variables.

Note: You will need Vault to be installed with an ADP-KM license add-on.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.


# Setup Vault

In [ ]:
# For this demo, we will be passing our doormat credentials as environment variables.
# For non-doormat scenarios, use your configured IAM programmatic credentials in the secret sync configuration later on.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/Documents/demos/vault/vault.hclic)

# Refresh Vault docker image with latest version
docker pull hashicorp/vault-enterprise

# Run Vault in docker in Dev mode with Enterprise license.
# We have set VAULT_LOG_LEVEL to trace for troubleshooting purposes.  This will allow you to view detailed information as you test.
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-e "VAULT_LOG_LEVEL=trace" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

In [ ]:
# Verify that Vault is running
docker ps

# Key Management Secrets Engine for AWS KMS

## Setup Vault KMSE for AWS KMS

In [ ]:
# Enable the Key Management secrets engine
vault secrets enable -path=kms-aws keymgmt

In [ ]:
# For HashiCorp staff only.  Log in via doormat and populate your AWS credentials into your environment variables.
doormat login -f && eval $(doormat aws export --role $(doormat aws list | grep -m 1 role))

# Make sure the AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, and AWS_SESSION_TOKEN environment variables are populated
echo
echo "AWS_ACCESS_KEY_ID: ${AWS_ACCESS_KEY_ID}"
echo "AWS_SECRET_ACCESS_KEY: ${AWS_SECRET_ACCESS_KEY}"
echo "AWS_SESSION_TOKEN: ${AWS_SESSION_TOKEN}"

export AWS_REGION=ap-southeast-1

In [ ]:
# Create KMS Provider 

# KMSE must be configured with credentials that have sufficient permissions to manage keys in an AWS KMS region. 
# The authentication parameters are described in the credentials section of the API documentation. 
# https://developer.hashicorp.com/vault/api-docs/secret/key-management/awskms#credentials
# The authentication parameters will be set with the following order of precedence:
# The minimum AWS KMS permissions for the IAM principal is listed here https://developer.hashicorp.com/vault/docs/secrets/key-management/awskms#authentication

vault write kms-aws/kms/demo-aws \
    provider="awskms" \
    key_collection="ap-southeast-1" \
    credentials=access_key="ASIA..." \
    credentials=secret_key="AbCdEf..." \
    credentials=session_token="AbCdEf..."



In [ ]:
# Write keys

# to do: set deletion_allowed as true

vault write kms-aws/key/aes256-gcm96-1 type="aes256-gcm96"
vault write kms-aws/key/aes256-gcm96-2 type="aes256-gcm96"

In [ ]:
# Read keys

vault read -format=json kms-aws/key/aes256-gcm96-1 | jq
vault read -format=json kms-aws/key/aes256-gcm96-2 | jq

## Distribute keys to AWS KMS

In [ ]:
# Distribute keys to AWS KMS

vault write kms-aws/kms/demo-aws/key/aes256-gcm96-1 \
    purpose="encrypt,decrypt" \
    protection="hsm"

vault write kms-aws/kms/demo-aws/key/aes256-gcm96-2 \
    purpose="encrypt,decrypt" \
    protection="hsm"


In [ ]:
# Verify keys have been distributed to AWS KMS
# You can also verify the keys have been distributed in the AWS Console
# Verify the key version and permitted operations (such as 'sign' for 'ecdsa-p256')

vault list kms-aws/kms/demo-aws/key/

## Rotate keys

In [ ]:
# Rotate the key in Vault

vault write -f kms-aws/key/aes256-gcm96-1/rotate

In [ ]:
# Confirm key rotation
# You can also check in the AWS Console that the key has a different version

 vault read -format=json kms-aws/key/aes256-gcm96-1 | jq '.data.latest_version'

# Key Management Secrets Engine for Azure Key Vault

You must configure an Azure Key Vault instance and assign an access policy that provides the key management policy to a service principal. 

You then need to generate a credential that Vault will use to connect to and manage the Key Vault.

[How to: Use the portal to create an Azure AD application and service principal that can access resources](https://learn.microsoft.com/en-us/entra/identity-platform/howto-create-service-principal-portal)

[Assign a Key Vault access policy using the Azure portal](https://docs.microsoft.com/en-us/azure/key-vault/general/assign-access-policy-portal)

Take note of the key vault name to use when you configure Vault later. eg. `learn-keyvault-52246b`

<img src="images/vault-demo-key-management-kms-akv.png">

## Setup Vault KMSE for Azure Key Vault

In [ ]:
# Enable the Key Management secrets engine
vault secrets enable -path=kms-azure keymgmt

You will need 4 pieces of information to configure the key vault instance

1. A key collection, which corresponds to the name of the key vault instance in Azure. eg. `learn-keyvault-52246b`
2. Azure client ID credential, that can also be specified with AZURE_CLIENT_ID environment variable.
3. Azure client secret credential, that can also be specified with AZURE_CLIENT_SECRET environment variable.
4. Azure tenant ID credential, that can also be specified with AZURE_TENANT_ID environment variable.


In [ ]:
# Create KMS Provider 

vault write kms-azure/kms/demo-azure \
    key_collection="jane-demo" \
    provider="azurekeyvault" 
    credentials=client_id="AbCdEf..." \
    credentials=client_secret="AbCdEf..." \
    credentials=tenant_id="AbCdEf..."

Based on the [Compatibility](https://developer.hashicorp.com/vault/docs/secrets/key-management#compatibility) section of the documentation, Azure Key Vault currently supports use of RSA-2048, RSA-3072, and RSA-4096 key types.

In [ ]:
# Write a pair of RSA-2048 and RSA-4096 keys

vault write kms-azure/key/rsa-2048 type="rsa-2048"
vault write kms-azure/key/rsa-4096 type="rsa-4096"

In [ ]:
# Read keys

vault read -format=json kms-azure/key/rsa-2048 | jq
vault read -format=json kms-azure/key/rsa-4096 | jq

## Distribute Keys to Azure Key Vault

In [ ]:
# Here you are instructing Vault to distribute the key and specify that its purpose is only to encrypt and decrypt. 
# The protection type is dependent on the cloud provider and the value is either hsm or software. 
# In the case of Azure, you specify hsm for the protection type. 
# The key will be securely delivered to the key vault instance according to the Azure Bring Your Own Key (BYOK) specification.
# https://learn.microsoft.com/en-us/azure/key-vault/keys/byok-specification

vault write kms-azure/kms/demo-azure/key/rsa-2048 \
    purpose="encrypt,decrypt" \
    protection="hsm"

vault write kms-azure/kms/demo-azure/key/rsa-4096 \
    purpose="sign" \
    protection="hsm"


In [ ]:
# Verify keys have been distributed to Azure Key Vault
# You can also verify the keys have been distributed in the Azure Console
# Verify the key version and permitted operations (such as 'sign' for 'rsa-4096')

vault list kms-azure/kms/demo-azure/key/

## Rotate keys

In [ ]:
# Rotate the key in Vault

vault write -f kms-azure/key/rsa-4096/rotate

In [ ]:
# Confirm key rotation
# You can also check in the Azure Console that the key has a different version

 vault read -format=json kms-azure/key/rsa-4096 | jq '.data.latest_version'

# Key Management Secrets Engine for GCP KMS

You must configure a Google Cloud project, enable billing for the project, and enable the required Cloud KMS API.

You must then create a KMS keyring and note its resource name. The example used in this tutorial is projects/learn-kmse/locations/global/keyRings/learn-keyring.

You must also create a service account, and assign it the proper roles in the project to manage Cloud KMS keys. The minimum roles required are as follows:

- cloudkms.cryptoKeys.create
- cloudkms.cryptoKeys.update
- cloudkms.importJobs.create
- cloudkms.importJobs.get
- cloudkms.cryptoKeyVersions.list
- cloudkms.cryptoKeyVersions.destroy
- cloudkms.cryptoKeyVersions.update
- cloudkms.cryptoKeyVersions.create

You then need to generate a JSON based credential file for this service account that Vault will use to connect to and manage the Cloud KMS instance. You can also specify the credentials using the GOOGLE_CREDENTIALS environment variable or [application default credentials](https://cloud.google.com/docs/authentication/production). Consult the [Key Management Secrets Engine documentation](https://developer.hashicorp.com/vault/docs/secrets/key-management/gcpkms) for more details.

<img src="images/vault-demo-key-management-kms-gcp.png">

## Setup Vault KMSE for GCP KMS

In [ ]:
# Enable the Key Management secrets engine
vault secrets enable -path=kms-gcp keymgmt

You will need 3 pieces of information to configure the Cloud KMS instance

1. A key collection, which refers to the [resource ID](https://cloud.google.com/kms/docs/resource-hierarchy#retrieve_resource_id) of an existing GCP Cloud KMS key ring; this value cannot be changed after creation.
2. Credentials file to use for authentication with GCP Cloud KMS. Supplying values for this parameter is optional, as credentials may also be specified as the `GOOGLE_CREDENTIALS` environment variable or default application credentials.

Set your google credentials before running this
- Ref: https://cloud.google.com/iam/docs/service-accounts-create
1) Enable IAM
2) Create Service Account - IAM & Admin > Service Accounts.  Click "CREATE SERVICE ACCOUNT" and give a name.  Copy the email of the created service account.
3) Click into service account, click on KEYS tab > ADD KEY > Create new key.  Select JSON for key type and click CREATE.
4) Download the json file to this folder and rename to gcp-kms-sync-service-account-credential.json
5) Give permissions to the service account - IAM & Admin > IAM.  Click on "GRANT ACCESS".  Paste the copied email for principals and assign the Secrets Manager Admin role.


In [ ]:
# Before we set up the KMS secrets engine with gcpkms, we need to place the GCP Service Account credential file inside the docker container running Vault
# According to the API documentation, "The key file must be readable on the host that Vault server is running on."
# https://developer.hashicorp.com/vault/api-docs/secret/key-management/gcpkms#service_account_file

docker cp gcp-kms-service-account-credential.json vault-enterprise:/gcp-kms-service-account-credential.json

In [ ]:
# Create KMS Provider

vault write kms-gcp/kms/demo-gcp \
    provider="gcpckms" \
    key_collection="projects/project-name/locations/global/keyRings/keyring-name" \
    credentials=service_account_file="gcp-kms-service-account-credential.json"

In [ ]:
# Write keys 

vault write kms-gcp/key/aes256-gcm96 type="aes256-gcm96"
vault write kms-gcp/key/ecdsa-p256 type="ecdsa-p256"

In [ ]:
# Read keys

vault read -format=json kms-gcp/key/aes256-gcm96 | jq
vault read -format=json kms-gcp/key/ecdsa-p256 | jq

## Distribute Keys to GCP KMS

In [ ]:
# Here you are instructing Vault to distribute the key and specify that its purpose is only to encrypt and decrypt. 
# The protection type is dependent on the cloud provider and the value is either hsm or software. 
# In the case of GCP, you specify hsm for the protection type. 

vault write kms-gcp/kms/demo-gcp/key/aes256-gcm96 \
    purpose="encrypt,decrypt" \
    protection="hsm"


vault write kms-gcp/kms/demo-gcp/key/ecdsa-p256 \
    purpose="sign" \
    protection="hsm"


In [ ]:
# Verify keys have been distributed to GCP KMS
# You can also verify the keys have been distributed in the GCP Console
# Verify the key version and permitted operations (such as 'sign' for 'ecdsa-p256')

vault list kms-gcp/kms/demo-gcp/key/

## Rotate keys

In [ ]:
# Rotate the key in Vault

vault write -f kms-gcp/key/aes256-gcm96/rotate

In [ ]:
# Confirm key rotation
# You can also check in the GCP Console that the key has a different version

 vault read -format=json kms-gcp/key/aes256-gcm96 | jq '.data.latest_version'

# Cleanup

In [ ]:
# Cleanup AWS KMS

vault delete kms-aws/kms/demo-aws/key/aes256-gcm96-1 
vault delete kms-aws/kms/demo-aws/key/aes256-gcm96-2 

In [ ]:
# Cleanup Azure Key Vault

vault delete kms-azure/kms/demo-azure/key/rsa-2048
vault delete kms-azure/kms/demo-azure/key/rsa-4096

In [ ]:
# Cleanup GCP KMS

vault delete kms-azure/kms/demo-azure/key/rsa-2048
vault delete kms-azure/kms/demo-azure/key/rsa-4096

In [ ]:
# Delete Vault

docker stop vault-enterprise

# 